In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN

# 데이터 로드 및 결측치 처리
data = pd.read_csv('AAME.csv')
openValues = data[['Open']]

openValues = openValues.replace(0.0, np.nan)  # 0.0: 결측치
openValues = openValues.dropna()   
print(openValues.head())
print(data)

     Open
66   3.80
71   3.50
72   3.50
118  4.60
194  5.25
             Date  Open  High   Low  Close  Adj Close  Volume
0      1980-03-17  0.00  4.05  3.85   3.85   3.213167   15000
1      1980-03-18  0.00  4.00  3.80   3.80   3.171437   10200
2      1980-03-19  0.00  4.05  3.85   3.85   3.213167   33500
3      1980-03-20  0.00  4.00  3.80   3.80   3.171437    8700
4      1980-03-21  0.00  3.95  3.75   3.75   3.129709   12700
...           ...   ...   ...   ...    ...        ...     ...
10093  2020-03-26  2.29  2.29  2.08   2.28   2.280000    1600
10094  2020-03-27  2.25  2.25  2.21   2.25   2.250000     500
10095  2020-03-30  2.25  2.25  2.13   2.13   2.130000     400
10096  2020-03-31  2.11  2.11  2.11   2.11   2.110000     300
10097  2020-04-01  2.11  2.15  2.11   2.15   2.150000     600

[10098 rows x 7 columns]


In [11]:
# 데이터 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(openValues)

# 데이터 분할
TEST_SIZE = 2000
train_data = scaled[:-TEST_SIZE]
test_data = scaled[-TEST_SIZE:]

# 샘플 생성 함수
def make_sample(sampleD, window):
    train = []
    target = []
    for i in range(len(sampleD)-window):
        train.append(sampleD[i:i+window])
        target.append(sampleD[i+window])
    return np.array(train), np.array(target)

WINDOW_SIZE = 10
X_train, y_train = make_sample(train_data, WINDOW_SIZE)
X_train = X_train.reshape(-1, WINDOW_SIZE, 1)

X_test, y_test = make_sample(test_data, WINDOW_SIZE)
X_test = X_test.reshape(-1, WINDOW_SIZE, 1)

In [13]:
# LSTM 모델
lstm_model = Sequential()
lstm_model.add(LSTM(16, input_shape=(WINDOW_SIZE, 1), activation='tanh'))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# RNN 모델
rnn_model = Sequential()
rnn_model.add(SimpleRNN(16, input_shape=(WINDOW_SIZE, 1), activation='tanh'))
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# 모델 학습
lstm_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
rnn_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

# 예측
lstm_pred = lstm_model.predict(X_test)
rnn_pred = rnn_model.predict(X_test)

# 역스케일링
lstm_pred = scaler.inverse_transform(lstm_pred)
rnn_pred = scaler.inverse_transform(rnn_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# MSE 계산
lstm_mse = mean_squared_error(y_test_inv, lstm_pred)
rnn_mse = mean_squared_error(y_test_inv, rnn_pred)

# 결과 출력
print(f"[LSTM] Mean Squared Error: {lstm_mse:.4f}")
print(f"[RNN] Mean Squared Error: {rnn_mse:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
[LSTM] Mean Squared Error: 0.0292
[RNN] Mean Squared Error: 0.0120
